# **LangGraph Chatbot with Long-Term Memory, Agentic Behavior, and Real-Time Streaming**
This notebook demonstrates a LangGraph-based chatbot with:
- **Long-term memory** using a vector store.
- **Agentic behavior** via conditional tool execution.
- **Real-time streaming** for responses.

Each step is implemented in separate cells below.

In [ ]:
!pip install -U langgraph langchain-openai langchain-community tiktoken

In [ ]:
import getpass
import os

def set_env(var: str):
"
                               "    if not os.environ.get(var):
"
                               "        os.environ[var] = getpass.getpass(f'Enter {var}: ')

"
                               "# Set your OpenAI API key
"
                               "set_env('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AnyMessage
from langchain_core.tools import tool
from typing import List, Annotated
import operator

from langgraph.prebuilt.chat_agent_executor import AgentState

In [ ]:
# Using AgentState directly
MyAgentState = AgentState

In [ ]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def search(query: str) -> str:
    """A simple search tool that returns a mocked weather report."""
    if 'sf' in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

tools_list = [search]
tools = {t.name: t for t in tools_list}
model = model.bind_tools(list(tools.values()))

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import uuid
import os

# Initialize FAISS vector store with persistence
embeddings = OpenAIEmbeddings()
FAISS_INDEX_PATH = os.getenv('FAISS_INDEX_PATH', './faiss_index')
try:
    if os.path.exists(FAISS_INDEX_PATH):
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
    else:
        # Create empty FAISS index with dummy text
        vector_store = FAISS.from_texts(['dummy'], embeddings)
        vector_store.save_local(FAISS_INDEX_PATH)
except Exception:
    # Fallback: create new index without persistence
    vector_store = FAISS.from_texts(['dummy'], embeddings)

@tool
def upsert_memory(content: str, context: str) -> str:
    memory_id = str(uuid.uuid4())
    document = Document(page_content=content, id=memory_id, metadata={'context': context})
    vector_store.add_documents([document])
    # Save to disk for persistence
    try:
        vector_store.save_local(FAISS_INDEX_PATH)
    except Exception:
        pass  # Continue even if save fails
    return f'Memory stored: {content}'

@tool
def search_memory(query: str) -> List[str]:
    docs = vector_store.similarity_search(query, k=3)
    return [doc.page_content for doc in docs]

for t in [upsert_memory, search_memory]:
    tools[t.name] = t
model = model.bind_tools(list(tools.values()))

In [ ]:
def load_memories(state: MyAgentState):
    from langchain_core.messages import get_buffer_string
    convo = get_buffer_string(state['messages'])
    retrieved = search_memory.invoke({'query': convo})
    memories_str = '\n'.join(retrieved) if retrieved else 'No relevant memories.'
    mem_message = SystemMessage(content=f'<memories>\n{memories_str}\n</memories>')
    return {'messages': [mem_message]}

In [ ]:
def call_llm(state: MyAgentState):
    messages = state['messages']
    response = model.invoke(messages)
    return {'messages': [response]}

def execute_function(state: MyAgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        if t['name'] not in tools:
            result = 'Error: Tool not found.'
        else:
            result = tools[t['name']].invoke(t['args'])
        results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
    return {'messages': results}

def exists_function_calling(state: MyAgentState):
    return len(state['messages'][-1].tool_calls) > 0

In [ ]:
checkpointer = MemorySaver()
graph_builder = StateGraph(MyAgentState)
graph_builder.add_node('load_memories', load_memories)
graph_builder.add_node('llm', call_llm)
graph_builder.add_node('tools', execute_function)

graph_builder.add_edge(START, 'load_memories')
graph_builder.add_edge('load_memories', 'llm')
graph_builder.add_conditional_edges('llm', exists_function_calling, {True: 'tools', False: END})
graph_builder.add_edge('tools', 'llm')
graph_builder.set_entry_point('load_memories')
graph = graph_builder.compile(checkpointer=checkpointer, store=vector_store)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print('Graph visualization not available:', e)

In [ ]:
initial_messages = [HumanMessage(content='What is the weather in SF?')]
config = {'configurable': {'thread_id': '1'}}

for event in graph.stream({'messages': initial_messages}, config, stream_mode='values'):
    print(event['messages'][-1].content)

In [ ]:
followup_messages = [HumanMessage(content='And what about New York?')]

for event in graph.stream({'messages': followup_messages}, config, stream_mode='values'):
    print(event['messages'][-1].content)